# Test with 3-D input

In [2]:
import sys, os
# get the parent directory of this notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from learning_loop.active_learning import active_learning_loop
# 
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar
from scipy.optimize import differential_evolution

In [10]:
def f(x):
    """
    3D nonlinear function with a global minimum near (0.3, 0.6, 0.9).
    x is expected to be an array-like of shape (3,)
    """
    x = np.asarray(x)
    return (x[0] - 0.3)**2 + (x[1] - 0.6)**2 + (x[2] - 0.9)**2 \
           + 0.5 * np.sin(5 * x[0]) * np.cos(3 * x[1])
X = np.random.rand(1000000, 3)
Y = np.array([f(x) for x in X])
idx = np.argmin(Y)
print("Brute Force with random search:", X[idx], Y[idx])
#
bounds = [(0,1), (0,1), (0,1)]  # 3-D box
result = differential_evolution(f, bounds, seed=42, polish=True)
print("Global minimum via differential_evolution:")
print(f"  x* = {result.x}")
print(f"  f(x*) = {result.fun:.6f}")

Brute Force with random search: [0.31189479 0.90555981 0.89792226] -0.36199545532496846
Global minimum via differential_evolution:
  x* = [0.31204695 0.90674098 0.89999995]
  f(x*) = -0.362004


In [33]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# Neural network configuration
nn_cfg = {
    "hidden_layers": [64, 32, 16],
    "activation": "relu",     # try "tanh" for smoother landscapes
    "dropout": 0.1,
    "epochs": 300,
    "lr": 1e-3,
}
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=100,
    M=8,                      # ensemble size
    model_type="nn",
    nn_config=nn_cfg,
    bootstrap=False,            # helps uncertainty early on
    acquisition="LCB",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Neural Network)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.1590
[001/100] acq=LCB, x_next=[0.7888, 0.0128, 0.0305], y_next=0.9804 (new), best_y=0.9804
[002/100] acq=LCB, x_next=[0.9465, 0.0022, 0.0519], y_next=0.9947 (new), best_y=0.9804
[003/100] acq=LCB, x_next=[0.892 , 0.0609, 0.021 ], y_next=0.9376 (new), best_y=0.9376
[004/100] acq=LCB, x_next=[0.9642, 0.0521, 0.0863], y_next=0.9125 (new), best_y=0.9125
[005/100] acq=LCB, x_next=[0.8463, 0.0717, 0.0566], y_next=0.8558 (new), best_y=0.8558
[006/100] acq=LCB, x_next=[0.7531, 0.8106, 0.1546], y_next=1.0268 (new), best_y=0.8558
[007/100] acq=LCB, x_next=[0.7667, 0.3465, 0.0127], y_next=0.9078 (new), best_y=0.8558
[008/100] acq=LCB, x_next=[0.7014, 0.2803, 0.0007], y_next=0.9530 (new), best_y=0.8558
[009/100] acq=LCB, x_next=[0.9471, 0.2141, 0.0046], y_next=0.9690 (new), best_y=0.8558
[010/100] acq=LCB, x_next=[0.6988, 0.2678, 0.0264], y_next=0.9128 (new), best_y=0.8558
[011/100] acq=LCB, x_next=[0.657 , 0.2498, 0.0342], y_next=0.9474 (new), be

In [30]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=200,
    M=8,                      # ensemble size
    model_type="poly",
    bootstrap=False,            # helps uncertainty early on
    acquisition="LCB",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Polynomial)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.1590
[001/200] acq=LCB, x_next=[0.8925, 0.2842, 0.0174], y_next=0.9110 (new), best_y=0.9110
[002/200] acq=LCB, x_next=[0.9902, 0.8944, 0.0503], y_next=1.7207 (new), best_y=0.9110
[003/200] acq=LCB, x_next=[0.9912, 0.0063, 0.3705], y_next=0.6254 (new), best_y=0.6254
[004/200] acq=LCB, x_next=[0.9645, 0.0205, 0.7924], y_next=0.2930 (new), best_y=0.2930
[005/200] acq=LCB, x_next=[0.9435, 0.1009, 0.9513], y_next=0.1885 (new), best_y=0.1885
[006/200] acq=LCB, x_next=[0.9035, 0.0678, 0.9485], y_next=0.1694 (new), best_y=0.1694
[007/200] acq=LCB, x_next=[0.7326, 0.0201, 0.9903], y_next=0.2830 (new), best_y=0.1694
[008/200] acq=LCB, x_next=[0.907 , 0.0615, 0.9099], y_next=0.1747 (new), best_y=0.1694
[009/200] acq=LCB, x_next=[0.8234, 0.8026, 0.9815], y_next=0.6291 (new), best_y=0.1694
[010/200] acq=LCB, x_next=[0.8121, 0.2116, 0.8618], y_next=0.0945 (new), best_y=0.0945
[011/200] acq=LCB, x_next=[0.7834, 0.3009, 0.9635], y_next=0.1103 (new), be

In [32]:
X0 = np.array([[0.1, 0.1, 0.1]])        # initial single observation
Y0 = np.array([f(X0[0])])               # corresponding function value
# Candidate pool:  1000 random samples in [0,1]^3
X_grid = np.random.rand(1000, 3)
# active learning loop
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=500,
    M=8,                      # ensemble size
    model_type="tree",
    bootstrap=False,            # helps uncertainty early on
    acquisition="EI",          # try "LCB" too for comparison
    patience=50,
    min_improve_pct=0.001,      # run until all candidates evaluated
)

# retrieve data for plotting
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])   # Y values evaluated each iteration
best_y = np.array([h[3] for h in history])   # running best values
best_final = np.min(best_y)
#
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters + 1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=result.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {result.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Random Forest)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white",
    width=800,
    height=500
)
fig.show()


Starting active learning run — initial best_y = 1.1590
[001/500] acq=EI, x_next=[0.6397, 0.7581, 0.9229], y_next=0.1594 (new), best_y=0.1594
[002/500] acq=EI, x_next=[0.1064, 0.9351, 0.7617], y_next=-0.0704 (new), best_y=-0.0704
[003/500] acq=EI, x_next=[0.3098, 0.8563, 0.1389], y_next=0.2250 (new), best_y=-0.0704
[004/500] acq=EI, x_next=[1.5006e-01, 9.7538e-01, 8.7635e-04], y_next=0.6388 (new), best_y=-0.0704
[005/500] acq=EI, x_next=[0.3922, 0.9114, 0.3552], y_next=-0.0223 (new), best_y=-0.0704
[006/500] acq=EI, x_next=[0.217 , 0.8894, 0.8485], y_next=-0.3002 (new), best_y=-0.3002
[007/500] acq=EI, x_next=[0.8736, 0.9005, 0.8895], y_next=0.8452 (new), best_y=-0.3002
[008/500] acq=EI, x_next=[0.7537, 0.9043, 0.9276], y_next=0.5658 (new), best_y=-0.3002
[009/500] acq=EI, x_next=[0.8015, 0.4815, 0.0914], y_next=0.8715 (new), best_y=-0.3002
[010/500] acq=EI, x_next=[0.6704, 0.9769, 0.949 ], y_next=0.3838 (new), best_y=-0.3002
[011/500] acq=EI, x_next=[0.4779, 0.8759, 0.277 ], y_next=0.1